In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tickerData import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = getTickerData(ticker="btc-usd", period="max", interval="1d")
df.drop(columns={'Adj Close'}, inplace=True)
df.rename(columns={'Date':'date', 'Open':'open', 'High':'high','Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)
df = produce_prediction(df, 1, dropna=True)
df

[*********************100%***********************]  1 of 1 completed


,date,open,high,low,close,volume,pred
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,1.0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,1.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,0.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,1.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,0.0
...,...,...,...,...,...,...,...
3097,2023-03-11,20187.876953,20792.525391,20068.660156,20632.410156,30180288176,0.0
3098,2023-03-12,20628.029297,22185.031250,20448.806641,22163.949219,29279035521,0.0
3099,2023-03-13,22156.406250,24550.837891,21918.199219,24197.533203,49466362688,0.0
3100,2023-03-14,24201.765625,26514.716797,24081.183594,24746.074219,54622230164,1.0


In [11]:
features = StandardScaler().fit_transform(df[df.columns.difference(['pred', 'date'])])
target = df['pred']

X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
x_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
x_train

array([[[-0.78619022],
        [-0.7852786 ],
        [-0.78650826],
        [-0.7850883 ],
        [-0.83379794]],

       [[-0.78823951],
        [-0.78596577],
        [-0.78903128],
        [-0.78564912],
        [-0.83312067]],

       [[-0.79008632],
        [-0.78772863],
        [-0.79086474],
        [-0.78768939],
        [-0.83294732]],

       ...,

       [[ 1.36835236],
        [ 1.37731605],
        [ 1.37175864],
        [ 1.42244882],
        [ 0.88319199]],

       [[ 1.27685159],
        [ 1.31422531],
        [ 1.29460283],
        [ 1.36811125],
        [ 1.07386389]],

       [[ 1.29709461],
        [ 1.2476349 ],
        [ 1.28734513],
        [ 1.27553192],
        [ 1.11875938]]])

In [13]:
lst = keras.Sequential() # initializing model

# input layer and LSTM layer with 50 neurons
lst.add(layers.LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
# outpute layer with sigmoid activation
lst.add(layers.Dense(1, activation='sigmoid'))

# defining loss function, optimizer, metrics and then compiling model
lst.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

lst.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 5, 50)             10400     
                                                                 
 dense_1 (Dense)             (None, 5, 1)              51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = lst.fit(x_train, Y_train, epochs=100, batch_size=16,validation_split=0.2)

Epoch 1/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6888 - accuracy: 0.5463 - val_loss: 0.7076 - val_accuracy: 0.5139
Epoch 2/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6888 - accuracy: 0.5464 - val_loss: 0.7103 - val_accuracy: 0.5054
Epoch 3/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6887 - accuracy: 0.5450 - val_loss: 0.7130 - val_accuracy: 0.5050
Epoch 4/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6888 - accuracy: 0.5464 - val_loss: 0.7137 - val_accuracy: 0.5014
Epoch 5/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6886 - accuracy: 0.5455 - val_loss: 0.7158 - val_accuracy: 0.5030
Epoch 6/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6886 - accuracy: 0.5455 - val_loss: 0.7171 - val_accuracy: 0.5014
Epoch 7/100
124/124 [==============================] - 0s 3ms/step - loss: 0.6889 - accuracy: 0.5461 - val_loss: 0.7194 - val_accuracy: 0.5006

In [16]:
x_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

# predicting target attribute on testing dataset
test_results = lst.evaluate(x_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

20/20 [==============================] - 0s 1ms/step - loss: 1.4612 - accuracy: 0.5137
Test results - Loss: 1.4612120389938354 - Accuracy: 51.36876106262207%
